# Synthesize a coloring book

In [1]:
import os
import numpy as np
import skimage
from PIL import Image
from PIL.Image import Image as PilImage

from diffusers import StableDiffusionPipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoImageProcessor, Swin2SRForImageSuperResolution
import torch
from torch import inf

import openai

# !pip install python-docx 
from docx import Document
from docx.shared import Inches, Mm
import re

2023-04-30 19:09:36.835197: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 19:09:39.780932: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-30 19:09:39.781020: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-30 19:09:39.781027: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

## Inputs

In [2]:
title = "The Zoo Animals Playing Sports Coloring Book"
topic = "anthropormophic zoo animals playing sports"
number_pages = 50
images_to_generate = 10
# upscale image
enhance_images = False
image_width = 768
image_height = 1024
image_dir = "./images"

## Define keys and load models

In [3]:
 # Azure Instance
openai.api_key = "
# your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_base =  "" 
openai.api_version = '2022-12-01' # this may change in the future
openai.api_type = 'azure'
#This will correspond to the custom name you chose for your deployment when you deployed a model. 
deployment_name='' 

def load_prompter():
    prompter_model = AutoModelForCausalLM.from_pretrained("microsoft/Promptist")
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"
    return prompter_model, tokenizer

prompter_model, prompter_tokenizer = load_prompter()

def load_upscaler():
    processor = AutoImageProcessor.from_pretrained("caidas/swin2SR-classical-sr-x2-64")
    model = Swin2SRForImageSuperResolution.from_pretrained("caidas/swin2SR-classical-sr-x2-64")
    return model, processor
    
upscale_model, upscale_processor = load_upscaler()

model_id = "dreamlike-art/dreamlike-photoreal-2.0"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

/home/gensyn/anaconda3/lib/python3.10/site-packages/transformers/modeling_utils.py:429: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(checkpoint_file, framework="pt") as f:
/home/gensyn/anaconda3/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/gensyn/anaconda3/lib/python3.10/site-packages/torch/storage.py:899: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage 

## Helper functions

In [4]:
# create a sketch for coloring based in an input image
def generate_coloring_page(input: PilImage) -> PilImage:
    # Convert to grayscale if needed
    if input.mode != "L":
        input = input.convert("L")
    np_image = np.asarray(input)
    
    # detect edges
    np_image = skimage.filters.sobel(np_image)
    # convert to 8 bpp
    np_image = skimage.util.img_as_ubyte(np_image)
    # Invert to get dark edges on a light background
    np_image = 255 - np_image
    # Improve the contrast
    np_image = skimage.exposure.rescale_intensity(np_image)
    
    return Image.fromarray(np_image)

# Create ideas for each desired page of the coloring book
def generate_coloring_book_page_descriptions(topic, number_pages):
    prompt = "Create "+str(number_pages)+" ideas for a coloring book on the topic of "+topic+"/n/nIdeas for coloring book topic:"
    response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.7,
            max_tokens=1500
        )
    result = response.choices[0].text.strip()
    return result, prompt

# Create fun facts about the image
def generate_fun_facts(topic):
    prompt = "Create fun facts about "+topic+"/n/nFun facts:"
    response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.7,
            max_tokens=1500
        )
    result = remove_after_create(response.choices[0].text.strip())
    return result

# Create fun facts about the image
def generate_summary(prompt: str):
    response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.5,
            max_tokens=1500
        )
    result = response.choices[0].text.strip()
    return result

# parse a string to list based on return characters
def parse_string(input_string: str) -> list:
    result = []
    lines = input_string.split('\n')
    for line in lines:
        if not 'Create ' in line and not line == "":
            result.append(remove_number_period(line.strip()))
    return result

def remove_after_create(s: str) -> str:
    keyword = "Create "
    index = s.find(keyword)
    
    if index == -1:
        # If "Create " is not found, return the original string
        return s
    
    # Find the end index of the keyword
    end_index = index + len(keyword)
    
    # Return the string up to the end of the keyword
    return s[:end_index]

def remove_number_period(s: str) -> str:
    # Use a regular expression to match a number followed by a period and a space
    pattern = r'^\d+\.\s'
    
    # Remove the matched pattern and return the modified string
    return re.sub(pattern, '', s).strip()

def generate(plain_text: str) -> str:
    input_ids = prompter_tokenizer(plain_text.strip()+" Rephrase:", return_tensors="pt").input_ids
    eos_id = prompter_tokenizer.eos_token_id
    outputs = prompter_model.generate(input_ids, do_sample=False, max_new_tokens=75, num_beams=8, num_return_sequences=8, eos_token_id=eos_id, pad_token_id=eos_id, length_penalty=-1.0)
    output_texts = prompter_tokenizer.batch_decode(outputs, skip_special_tokens=True)
    res = output_texts[0].replace(plain_text+" Rephrase:", "").strip()
    return res

def enhance_image(image):
    # prepare image for the model
    inputs = upscale_processor(image, return_tensors="pt")

    # forward pass
    with torch.no_grad():
        outputs = upscale_model(**inputs)

    # postprocess
    output = outputs.reconstruction.data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.moveaxis(output, source=0, destination=-1)
    output = (output * 255.0).round().astype(np.uint8)  # float32 to uint8
    
    return Image.fromarray(output)

def create_directory_if_not_exists(path:str):
    """
    Creates a directory at the specified path if it does not already exist.
    """
    if not os.path.exists(path):
        os.makedirs(path)
    return

## Workflow

In [5]:
# Create page ideas given the desired topic
page_ideas = generate_coloring_book_page_descriptions(topic, number_pages)
print(page_ideas)

('Anthropomorphic Zoo Animals Playing Sports\n\n1. Gorilla playing baseball \n2. Elephant playing football \n3. Giraffe playing volleyball \n4. Lion playing basketball \n5. Hippo playing tennis \n6. Zebra playing golf \n7. Rhino playing soccer \n8. Tiger playing hockey \n9. Monkey playing rugby \n10. Koala playing cricket \n11. Fox playing badminton \n12. Polar bear playing lacrosse \n13. Emu playing frisbee \n14. Meerkat playing dodgeball \n15. Alligator playing handball \n16. Cheetah playing squash \n17. Fox playing softball \n18. Ostrich playing field hockey \n19. Wallaby playing lacrosse \n20. Wombat playing croquet \n21. Pangolin playing ultimate frisbee \n22. Sloth playing bocce ball \n23. Badger playing disc golf \n24. Beaver playing basketball \n25. Antelope playing volleyball \n26. Armadillo playing baseball \n27. Capybara playing badminton \n28. Gibbon playing rugby \n29. Porcupine playing soccer \n30. Mongoose playing hockey \n31. Hedgehog playing cricket \n32. Wild boar pla

In [6]:
# parse the page ideas into a list for each page
pages = parse_string(page_ideas[0])
print(pages)

['Anthropomorphic Zoo Animals Playing Sports', 'Gorilla playing baseball', 'Elephant playing football', 'Giraffe playing volleyball', 'Lion playing basketball', 'Hippo playing tennis', 'Zebra playing golf', 'Rhino playing soccer', 'Tiger playing hockey', 'Monkey playing rugby', 'Koala playing cricket', 'Fox playing badminton', 'Polar bear playing lacrosse', 'Emu playing frisbee', 'Meerkat playing dodgeball', 'Alligator playing handball', 'Cheetah playing squash', 'Fox playing softball', 'Ostrich playing field hockey', 'Wallaby playing lacrosse', 'Wombat playing croquet', 'Pangolin playing ultimate frisbee', 'Sloth playing bocce ball', 'Badger playing disc golf', 'Beaver playing basketball', 'Antelope playing volleyball', 'Armadillo playing baseball', 'Capybara playing badminton', 'Gibbon playing rugby', 'Porcupine playing soccer', 'Mongoose playing hockey', 'Hedgehog playing cricket', 'Wild boar playing tennis', 'Camel playing golf', 'Hyena playing dodgeball', 'Jackal playing handball'

In [7]:
# create a summary
prompt = "Create a summary for a coloring book about "+topic
summary = generate_summary(prompt)
summary

'This fun and creative coloring book features anthropomorphic zoo animals playing a variety of sports! Kids will love coloring in the pages of friendly animals like monkeys, giraffes, zebras, and more as they play soccer, basketball, baseball, and other sports. With a variety of scenes and activities, this book is sure to provide hours of entertainment and creativity. Whether they’re coloring in the animals as they practice their skills or as they compete in a championship game, kids will love this exciting coloring book.'

In [8]:
# create fun facts and images for each page
fun_facts = []
images = []
count = 0
negative_prompt = "watermark, text, shading, gradient, shadows, transparency, noisy, blurred"
create_directory_if_not_exists(image_dir)
# iterate for each desired page
for page in pages:
    prompt = "photo, anthropormophic "+page
    # improve the image prompt with a model
    new_prompt = generate(prompt)
    # iterate for the desired number of images to choose from for each page
    for image_num in range(0, images_to_generate):
        # create an image based on the improved prompt
        image = pipe(new_prompt, guidance_scale=8, num_inference_steps=80, height=image_height, width=image_width).images[0]
        # enhance/upscale the generated image
        if enhance_images:
            enhanced_image = enhance_image(image)
            # create a coloring book image based on the generated image
            coloring_book_image = generate_coloring_page(enhanced_image)
        else:
            coloring_book_image = generate_coloring_page(image)
        # save the image with the format image_pageNumber_imageNumber as jpg
        coloring_book_image.save(image_dir+"/image_"+str(count)+"_"+str(image_num)+".jpg")
        image.save(image_dir+"/photo_"+str(count)+"_"+str(image_num)+".jpg")
    # add the fun facts for each page in a list
    fun_facts.append(remove_after_create(generate_fun_facts(page)))
    count = count + 1

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

...

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

In [11]:
# write to a word document
# create a new docx document object
document = Document()

# set the document size and margins
section = document.sections[0]
section.page_width = Inches(6.0)
section.page_height = Inches(9.0)
section.left_margin = Inches(0.375)
section.right_margin = Inches(0.375)
section.top_margin = Inches(0.5)
section.bottom_margin = Inches(0.5)

# Add a title page
document.add_heading(title, 0)
p = document.add_paragraph('A coloring book about '+topic)
p = document.add_paragraph(summary)

count = 0
for page in pages:
    document.add_page_break()
    document.add_heading(pages[count], level=1)
    document.add_picture(image_dir+"/image_"+str(count)+"_0.jpg", width=Inches(5.25))
    document.add_page_break()
    document.add_paragraph('Fun facts about '+pages[count], style='Intense Quote')
    document.add_paragraph(fun_facts[count])
    count += 1

document.save('demo.docx')